# **ETL (Extracción, Transformación y Carga)**

## **Importación de librerias**

In [221]:
import os
import os.path
import json
import numpy as np
import pandas as pd
import datetime as dt
import ast
from ast import literal_eval
import requests
import re
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
#!pip install googletrans
from googletrans import Translator
#!pip install pyspark
from pyspark.sql import SparkSession


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## **Ingesta de Datos**

Se desarrollo en Google Colab para que los dataset no arrojen error al momento de cargar en Github

In [3]:
# Directorio donde se encuentran los archivos CSV
dataset_dir = "/content/drive/MyDrive/Colab Notebooks/PI1/dataset"

# Lista de archivos en el directorio (ajusta según tus archivos)
files = ["credits.csv", "movies_dataset.csv"]

# Creamos una lista con las rutas completas de los archivos CSV
csv_files = [f'{dataset_dir}/{file}'
            for file in files
            if file.endswith('.csv')
            and os.path.isfile(os.path.join(dataset_dir, file))]

# Leemos los archivos CSV y creamos los DataFrames específicos
credits_csv = pd.read_csv(csv_files[0])
movies_csv = pd.read_csv(csv_files[1], low_memory=False)

print("Archivos CSV leídos correctamente:")

Archivos CSV leídos correctamente:


## **Tratamiento de datos**


### **Datafame ```credits_csv```**

In [4]:
# Imprimir las primeras filas
credits_csv.head()


,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [5]:
# Imprime las últimas 5 filas
credits_csv.tail()

,cast,crew,id
45471,"[{'cast_id': 0, 'character': '', 'credit_id': ...","[{'credit_id': '5894a97d925141426c00818c', 'de...",439050
45472,"[{'cast_id': 1002, 'character': 'Sister Angela...","[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de...",111109
45473,"[{'cast_id': 6, 'character': 'Emily Shaw', 'cr...","[{'credit_id': '52fe4776c3a368484e0c8387', 'de...",67758
45474,"[{'cast_id': 2, 'character': '', 'credit_id': ...","[{'credit_id': '533bccebc3a36844cf0011a7', 'de...",227506
45475,[],"[{'credit_id': '593e676c92514105b702e68e', 'de...",461257


In [6]:
# Información básica sobre el dataset: filas, columnas, tipo de datos y valores nulos.
credits_csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cast    45476 non-null  object
 1   crew    45476 non-null  object
 2   id      45476 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


In [7]:
# Cantidad de registros y columnas
entrada_original = credits_csv.shape[0]
credits_csv.shape

(45476, 3)

Valores nulos en el dataset original

In [8]:
# Cantidad de valores nulos de cada columna
conteo_valores_faltantes = credits_csv.isnull().sum()
columnas = credits_csv.shape[1]
conteo_valores_faltantes[0:columnas]

,0
cast,0
crew,0
id,0


**Examinación del dataset**

El dataset presenta 2 columnas en formato JSON: cast y crew.

In [9]:
# Acceder a la primera fila para verificar si las columnas contienen este formato.
credits_csv.loc[0]

,0
cast,"[{'cast_id': 14, 'character': 'Woody (voice)',..."
crew,"[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."
id,862


In [10]:
# Acceder a los registros de la fila 20 de la columna cast
credits_csv.cast.loc[20]

'[{\'cast_id\': 1, \'character\': \'Chili Palmer\', \'credit_id\': \'52fe448dc3a36847f809c6eb\', \'gender\': 2, \'id\': 8891, \'name\': \'John Travolta\', \'order\': 0, \'profile_path\': \'/ns8uZHEHzV18ifqA9secv8c2Ard.jpg\'}, {\'cast_id\': 2, \'character\': \'Harry Zimm\', \'credit_id\': \'52fe448dc3a36847f809c6ef\', \'gender\': 2, \'id\': 193, \'name\': \'Gene Hackman\', \'order\': 1, \'profile_path\': \'/qEKcmwc1XstymEniGuCs3KIsGfP.jpg\'}, {\'cast_id\': 3, \'character\': \'Karen Flores\', \'credit_id\': \'52fe448dc3a36847f809c6f3\', \'gender\': 1, \'id\': 14343, \'name\': \'Rene Russo\', \'order\': 2, \'profile_path\': \'/aaPzuXRHdyxamPMXqu70okPaqvB.jpg\'}, {\'cast_id\': 4, \'character\': \'Martin Weir\', \'credit_id\': \'52fe448dc3a36847f809c6f7\', \'gender\': 2, \'id\': 518, \'name\': \'Danny DeVito\', \'order\': 3, \'profile_path\': \'/zKuyzmKzPLG7RJo7lbbHjx6CCZc.jpg\'}, {\'cast_id\': 5, \'character\': "Ray \'Bones\' Barboni", \'credit_id\': \'52fe448dc3a36847f809c6fb\', \'gender\

Irregularidad de los datos (Data Entry)

Atributo ```ID```

In [11]:
# Arrays de los id
unicos_id = credits_csv.id.unique()
unicos_id.sort()
print(unicos_id)

[     2      3      5 ... 468343 468707 469172]


In [12]:
# Cantidad de veces que se repiten los id
credits_csv.id.value_counts().head()

# Hay registros duplicados.

,count
id,
141971,3
298721,2
9755,2
10991,2
99080,2


Se presentan registros duplicados, el siguiente paso será desanidar la información del dataframe para extraer únicamente la información valiosa.

Columnas ```cast``` y ```crew```

In [13]:
# Filtrar los registros con valores nulos (NaN)

# Fila en la columna cast
filtro_col_cast = credits_csv[credits_csv['cast'].str.contains('[]', regex = False)]
filtro_col_cast.head()
#filtro_col_cast.tail()

,cast,crew,id
137,[],"[{'credit_id': '52fe4ab0c3a368484e161d3d', 'de...",124639
240,[],"[{'credit_id': '52fe464ac3a36847f80f6d61', 'de...",43475
393,[],"[{'credit_id': '52fe4624c3a36847f80ef0a5', 'de...",42981
438,[],"[{'credit_id': '52fe448dc3a368484e029383', 'de...",24257
595,[],"[{'credit_id': '52fe4aacc3a368484e16115b', 'de...",124472


In [14]:
vacios_cast_data = filtro_col_cast.shape[0] # cantidad de registros de la columna cast
print(f'Se encontró {vacios_cast_data} registros NaN en la columna "cast"')

Se encontró 2418 registros NaN en la columna "cast"


In [15]:
# Fila en la columna crew
filtro_col_crew = credits_csv[credits_csv['crew'].str.contains('[]', regex = False)]
filtro_col_crew.head()
#filtro_col_crew.tail()

,cast,crew,id
189,"[{'cast_id': 4, 'character': 'Himself', 'credi...",[],56088
614,"[{'cast_id': 1, 'character': 'Grace Rhodes', '...",[],123505
635,"[{'cast_id': 0, 'character': 'Joachim Krippo',...",[],339428
661,[],[],318177
711,[],[],365371


Se encontraron estos datos '[ ]' que representan registros nulos o valores NaN.

In [16]:
vacios_crew_data = filtro_col_crew.shape[0] # cantidad de registros de la columna crew
print(f'Se encontró {vacios_crew_data} registros NaN en la columna "crew"')

Se encontró 771 registros NaN en la columna "crew"


# Desanidar la data

En las columnas cast y crew contiene registros en formato listas de diccionarios (una lista en donde cada elemento es un diccionario)

In [17]:
# Creación de una copia para no modificar el dataset original
copia_credits = credits_csv.copy()

In [18]:
# Convertir una cadena que tiene una lista en una lista de objetos para 'cast' y 'crew'
columna_credits = ['cast', 'crew']

In [19]:
# La función 'convertir_a_lista' convierte la cadena en una lista de objetos
def convertir_a_lista(valor):
    return ast.literal_eval(valor)      # retorna una lista creada a partir de una cadena.

# Iteración de cada columna de la lista de 'columna_credits'.
# Se le aplica la función 'convertir_a_lista' a cada valor de la columna utilizando el método apply
for columna in columna_credits:
    copia_credits[columna] = copia_credits[columna].apply(convertir_a_lista)

# Como resultado se asigna de vuelta a la columna del dataframe 'copia_credits'.
# Se visualiza las primeras filas del dataframe modificiado.
copia_credits.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': 'Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [20]:
# Visualizar los valores de la primera fila en la columna 'cast'
copia_credits.cast.loc[0][0]

{'cast_id': 14,
 'character': 'Woody (voice)',
 'credit_id': '52fe4284c3a36847f8024f95',
 'gender': 2,
 'id': 31,
 'name': 'Tom Hanks',
 'order': 0,
 'profile_path': '/pQFoyx7rp09CJTAb932F2g8Nlho.jpg'}

In [21]:
# Visualizar los valores de la fila 20 en la columna 'cast'
copia_credits.cast.loc[20][0]

{'cast_id': 1,
 'character': 'Chili Palmer',
 'credit_id': '52fe448dc3a36847f809c6eb',
 'gender': 2,
 'id': 8891,
 'name': 'John Travolta',
 'order': 0,
 'profile_path': '/ns8uZHEHzV18ifqA9secv8c2Ard.jpg'}

In [22]:
# Visualizar los valores de la primera fila en la columna 'crew'
copia_credits.crew.loc[0][0]

{'credit_id': '52fe4284c3a36847f8024f49',
 'department': 'Directing',
 'gender': 2,
 'id': 7879,
 'job': 'Director',
 'name': 'John Lasseter',
 'profile_path': '/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg'}

In [23]:
# Visualizar los valores de la fila 20 en la columna 'crew'
copia_credits.crew.loc[20][0]

{'credit_id': '52fe448dc3a36847f809c729',
 'department': 'Directing',
 'gender': 2,
 'id': 5174,
 'job': 'Director',
 'name': 'Barry Sonnenfeld',
 'profile_path': '/m8vIfbXI44Fd8VK7HSJRisYQjQc.jpg'}

In [24]:
# Conteo de profesiones dentro de la clave 'job' en la columna 'crew'
profesiones = pd.Series([d['job'] for d in copia_credits.crew[0]]).value_counts().sort_index()
print(profesiones.head(20))

ADR Editor                   1
ADR Voice Casting            1
Animation                   26
Animation Director           2
Art Direction                1
Assistant Editor             4
Assistant Sound Editor       4
CG Painter                   2
Casting Consultant           1
Character Designer           8
Color Timer                  1
Director                     1
Editor                       2
Editorial Coordinator        1
Editorial Manager            1
Executive Music Producer     1
Executive Producer           2
Foley Editor                 1
Layout                       2
Lighting Supervisor          2
Name: count, dtype: int64


Columna crew

In [25]:
# Función para obtener los nombres de los directores del atributo 'crew'
def obtener_nombres_directores(datos):
    if isinstance(datos, list):
        directores = list(filter(lambda x: x['job'] == 'Director', datos))
        return ', '.join(director['name'] for director in directores) if directores else np.nan
    else:
        return np.nan

copia_credits['director'] = copia_credits['crew'].apply(obtener_nombres_directores)
copia_credits.head()

,cast,crew,id,director
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862,John Lasseter
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844,Joe Johnston
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602,Howard Deutch
3,"[{'cast_id': 1, 'character': 'Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357,Forest Whitaker
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862,Charles Shyer


In [26]:
# Se verifica si hay cadenas vacías en el atributo "director"
copia_credits[copia_credits['director'] == '']

,cast,crew,id,director


In [27]:
# Mostrar un conteo de los directores previamente extraidos
copia_credits['director'].value_counts(dropna= False).head(5)

,count
director,
NaN,887
John Ford,63
Michael Curtiz,61
Alfred Hitchcock,52
Werner Herzog,52


Columna cast

In [28]:
def obtener_nombres_actores(datos):
    if isinstance(datos, list):
        actores = list(filter(lambda x: 'name' in x, datos))
        return ', '.join(actor['name'] for actor in actores) if actores else np.nan
    else:
        return np.nan

In [29]:
# Se crea una nueva columna 'actores' en el DataFrame "copia_credits" utilizando la función "obtener_nombres_actores"
copia_credits['actores'] = copia_credits['cast'].apply(obtener_nombres_actores)
copia_credits.head()

,cast,crew,id,director,actores
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862,John Lasseter,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney,..."
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844,Joe Johnston,"Robin Williams, Jonathan Hyde, Kirsten Dunst, ..."
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602,Howard Deutch,"Walter Matthau, Jack Lemmon, Ann-Margret, Soph..."
3,"[{'cast_id': 1, 'character': 'Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357,Forest Whitaker,"Whitney Houston, Angela Bassett, Loretta Devin..."
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862,Charles Shyer,"Steve Martin, Diane Keaton, Martin Short, Kimb..."


In [30]:
# Se verifica si hay cadenas vacías en el atributo "actores"
copia_credits[copia_credits['actores'] == '']

,cast,crew,id,director,actores


In [31]:
copia_credits['actores'] = copia_credits['cast'].apply(obtener_nombres_actores)

In [32]:
# Conteo de los actores
copia_credits['actores'].value_counts(dropna= False).head()

,count
actores,
NaN,2418
Georges Méliès,24
Louis Theroux,15
Mel Blanc,12
Jimmy Carr,9


Dataframe con los atributos: id, director y actores.

In [33]:
creditos = copia_credits[['id', 'director', 'actores']]
creditos.head()

,id,director,actores
0,862,John Lasseter,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney,..."
1,8844,Joe Johnston,"Robin Williams, Jonathan Hyde, Kirsten Dunst, ..."
2,15602,Howard Deutch,"Walter Matthau, Jack Lemmon, Ann-Margret, Soph..."
3,31357,Forest Whitaker,"Whitney Houston, Angela Bassett, Loretta Devin..."
4,11862,Charles Shyer,"Steve Martin, Diane Keaton, Martin Short, Kimb..."


In [34]:
# Determinar el total de los valores faltantes por cada columna.
conteo_valores_faltantes = creditos.isnull().sum()
columnas = creditos.shape[1]
conteo_valores_faltantes[0:columnas]

,0
id,0
director,887
actores,2418


In [35]:
nan_director = creditos['director'].isnull().sum()
print(f'''Columna crew:
     Cantidad de registros tipo string con listas vacías ('[]'): {vacios_crew_data}
     Cantidad de registros nulos: {nan_director}
     Cantidad de listas que no incluyen el nombre del director: {nan_director - vacios_crew_data}''')

Columna crew:
     Cantidad de registros tipo string con listas vacías ('[]'): 771
     Cantidad de registros nulos: 887
     Cantidad de listas que no incluyen el nombre del director: 116


In [36]:
nan_actores = creditos['actores'].isnull().sum()
print(f'''Columna cast:
     Cantidad de registros tipo string con listas vacías ('[]'): {vacios_cast_data}
     Cantidad de registros nulos: {nan_actores}
     Cantidad de listas que no incluyen el nombre del actor: {nan_actores - vacios_cast_data}''')

Columna cast:
     Cantidad de registros tipo string con listas vacías ('[]'): 2418
     Cantidad de registros nulos: 2418
     Cantidad de listas que no incluyen el nombre del actor: 0


Eliminar datos duplicados

In [37]:
# Verificar si hay datos duplicados en la columna 'id'
duplicados_id = creditos.duplicated(subset='id', keep=False).sum()
print(duplicados_id)

87


In [38]:
# ID que más se repiten en el atributo 'id'
creditos['id'].value_counts().head()

,count
id,
141971,3
298721,2
9755,2
10991,2
99080,2


In [39]:
# Verificación de las 3 apariciones del id 141971
id_duplicado = creditos[creditos['id'] == 141971]
id_duplicado

,id,director,actores
13261,141971,JP Siili,"Petteri Summanen, Ismo Kallio, Eppu Salminen, ..."
13375,141971,JP Siili,"Petteri Summanen, Ismo Kallio, Eppu Salminen, ..."
16764,141971,JP Siili,"Petteri Summanen, Ismo Kallio, Eppu Salminen, ..."


In [40]:
# Eliminación de id duplicados en el atributo 'id'
creditos = creditos.groupby('id', as_index=False).first()

In [41]:
# Verificar si quedaron valores duplicados.
valores_unicos = creditos['id'].nunique()
cantidad_filas = creditos.shape[0]
if valores_unicos == cantidad_filas:
    print("No hay duplicados en la columna 'id'")
else:
    print("Aún hay duplicados en la columna 'id'")

No hay duplicados en la columna 'id'


In [42]:
# Verificación con el ejemplo del id 141971
id_duplicado = creditos[creditos['id'] == 141971]
id_duplicado

,id,director,actores
33178,141971,JP Siili,"Petteri Summanen, Ismo Kallio, Eppu Salminen, ..."


In [43]:
print(f'Cantidad de registros con duplicados: {entrada_original}')
entrada_final = creditos.shape[0]
print(f'Cantidad de registros sin duplicados: {entrada_final}')
print(f'% de datos eliminados comparado a la cantidad de entrada orginal: {round(100 - (entrada_final * 100/ entrada_original), 4)}')

# la pérdida de data fue prácticamente nula

Cantidad de registros con duplicados: 45476
Cantidad de registros sin duplicados: 45432
% de datos eliminados comparado a la cantidad de entrada orginal: 0.0968


Dataset limpio

In [44]:
creditos.head()

,id,director,actores
0,2,Aki Kaurismäki,"Turo Pajala, Susanna Haavisto, Matti Pellonpää..."
1,3,Aki Kaurismäki,"Matti Pellonpää, Kati Outinen, Sakari Kuosmane..."
2,5,"Allison Anders, Alexandre Rockwell, Robert Rod...","Tim Roth, Antonio Banderas, Jennifer Beals, Ma..."
3,6,Stephen Hopkins,"Emilio Estevez, Cuba Gooding Jr., Denis Leary,..."
4,11,George Lucas,"Mark Hamill, Harrison Ford, Carrie Fisher, Pet..."


In [45]:
creditos.tail()

,id,director,actores
45427,465044,"Molly Smith, Maurice Smith","Karolina Antosik, Amelie Leroy, Tessa McGinn, ..."
45428,467731,Sidney Lumet,"Lloyd Bridges, Jack Warden, Rafael Campos, Rob..."
45429,468343,Jack Witikka,None
45430,468707,Hannaleena Hauru,"Inka Haapamäki, Rosa Honkonen, Tiitus Rantala,..."
45431,469172,Raúl Ruiz,"Ruben de Freitas, Teresa Madruga, Fernando Hei..."


# **Datafame ```movies_csv```**

In [46]:
# Imprimir las primeras filas
movies_csv.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [47]:
# Visualizar las últimas filas del dataset
movies_csv.tail()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
45461,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,NaN,0.0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0
45462,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0
45463,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0
45464,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,1917-10-21,0.0,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0
45465,False,NaN,0,[],NaN,461257,tt6980792,en,Queerama,50 years after decriminalisation of homosexual...,...,2017-06-09,0.0,75.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Queerama,False,0.0,0.0


In [48]:
# Información sobre el dataset
movies_csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [49]:
entrada_original = movies_csv.shape[0]
movies_csv.shape

(45466, 24)

In [50]:
# Cantidad de valores nulos de cada columna
conteo_valores_faltantes = movies_csv.isnull().sum()
columnas = movies_csv.shape[1]
conteo_valores_faltantes[0:columnas]

,0
adult,0
belongs_to_collection,40972
budget,0
genres,0
homepage,37684
id,0
imdb_id,17
original_language,11
original_title,0
overview,954


In [51]:
# % de datos faltantes respecto al total de los datos presentes en "movies_df"
total_cells = movies_csv.size
total_missing = movies_csv.isnull().sum().sum()
percent_missing = (total_missing / total_cells) * 100
print(f'{round(percent_missing, 2)} %')

9.67 %


Eliminación de columnas innecesarias

In [52]:
# Eliminar columnas video, imdb_id, adult, original_title, poster_path y homepage

# Columnas a eliminar
columnas_a_eliminar = ['adult', 'imdb_id', 'homepage', 'original_title', 'poster_path', 'video']

# Creamos un índice booleano para las columnas a eliminar
columnas_a_eliminar_idx = movies_csv.columns.isin(columnas_a_eliminar)

# Eliminamos las columnas utilizando el índice booleano
movies_df = movies_csv.drop(movies_csv.columns[columnas_a_eliminar_idx], axis=1)

# Verificación de la modificación
movies_csv.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [53]:
movies_df.shape

(45466, 18)

Buscar irregularidades

In [54]:
movies_csv.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

Atributo ```overview```

In [55]:
movies_df['overview'].value_counts(dropna= False).head()

,count
overview,
NaN,954
No overview found.,133
No Overview,7
,5
No movie overview available.,3


In [56]:
# Identificar las filas donde la columna 'overview' contiene solo espacios en blanco
# Se crea una máscara booleana para seleccionar estas filas
mask = movies_csv['overview'].str.strip() == ''  # Elimina espacios en blanco y compara con cada cadena vacía

# Reemplazar las cadenas vacías con valores nulos (NaN)
movies_csv.loc[mask, 'overview'] = np.nan

# Contar la frecuencia de cada valor único en la columna 'overview', incluyendo los valores nulos
counts = movies_csv['overview'].value_counts(dropna=False)

# Mostrar los 5 valores más comunes
counts.head()

,count
overview,
NaN,959
No overview found.,133
No Overview,7
Released,3
"King Lear, old and tired, divides his kingdom among his daughters, giving great importance to their protestations of love for him. When Cordelia, youngest and most honest, refuses to idly flatter the old man in return for favor, he banishes her and turns for support to his remaining daughters. But Goneril and Regan have no love for him and instead plot to take all his power from him. In a parallel, Lear's loyal courtier Gloucester favors his illegitimate son Edmund after being told lies about his faithful son Edgar. Madness and tragedy befall both ill-starred fathers.",3


Atributo ```original_language```

In [57]:
# Lenguajes que más se repiten
movies_csv['original_language'].value_counts(dropna= False)
# print(f'El lenguaje que más se repite es: {movies_df['original_language'].value_counts(dropna= False).idxmax()}')

,count
original_language,
en,32269
fr,2438
it,1529
ja,1350
de,1080
...,...
zu,1
qu,1
104.0,1


In [58]:
# Verificar si hay números
contiene_numeros = movies_csv['original_language'].str.contains(r'\d', na=False)
# str.contains(r'\d', na=False) expresión para buscar en toda la columna si hay algún número e
# incluir valores nulos no coincidentes, evitando que efecte el resultado.

movies_csv[contiene_numeros]['original_language']

,original_language
19730,104.0
29503,68.0
35587,82.0


In [59]:
# Reemplazar los datos por valores NaN
movies_csv.loc[contiene_numeros, 'original_language'] = np.nan

# Verificar las modificaciones
movies_csv[contiene_numeros].original_language

,original_language
19730,NaN
29503,NaN
35587,NaN


Atributo ```status ```

In [60]:
# Cuenta la frecuencia de cada valor único en la columna 'status', incluyendo los valores nulos
movies_df['status'].value_counts(dropna= False)

,count
status,
Released,45014
Rumored,230
Post Production,98
NaN,87
In Production,20
Planned,15
Canceled,2


Atributo ```tagline ```

In [61]:
# Reemplaza los guiones "-" con valores nulos (NaN) en la columna 'tagline' y cuenta la frecuencia de repetición de cada valor
movies_df['tagline'] = movies_df['tagline'].replace('-', np.nan)
movies_df['tagline'].value_counts()

,count
tagline,
Based on a true story.,7
Trust no one.,4
Be careful what you wish for.,4
Know Your Enemy,3
Who is John Galt?,3
...,...
A special force in a special kind of hell!,1
Play it. Sing it. Shout it. Feel it.,1
"If It's On TV, It Must Be The Truth.",1


Atributo ``` title ```

In [62]:
# Cuenta la frecuencia de cada título de película
movies_df['title'].value_counts(dropna= False)

,count
title,
Cinderella,11
Alice in Wonderland,9
Hamlet,9
Les Misérables,8
Beauty and the Beast,8
...,...
Cluny Brown,1
Babies,1
The Green Room,1


In [63]:
# Identificar valores vacios de la columna 'title' y elimina los espacios en blanco
movies_df.loc[movies_df['title'].str.strip() == '']

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count


In [64]:
movies_df.loc[movies_df['overview'].str.strip() == '']

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
4246,NaN,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",47596,en,,0.046139,[],"[{'iso_3166_1': 'US', 'name': 'United States o...",1999-01-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Rumored,NaN,Snow days,5.0,1.0
4538,NaN,0,"[{'id': 18, 'name': 'Drama'}]",49788,en,,0.684803,[],"[{'iso_3166_1': 'US', 'name': 'United States o...",1989-03-18,0.0,124.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Slaves of New York,4.8,6.0
18171,NaN,0,[],46770,en,,0.122466,[],"[{'iso_3166_1': 'AR', 'name': 'Argentina'}, {'...",1988-01-01,0.0,127.0,"[{'iso_639_1': 'es', 'name': 'Español'}, {'iso...",NaN,NaN,Sur,3.7,3.0
28408,NaN,0,"[{'id': 35, 'name': 'Comedy'}]",47110,en,,0.234066,[],"[{'iso_3166_1': 'GR', 'name': 'Greece'}]",2008-10-23,0.0,102.0,"[{'iso_639_1': 'el', 'name': 'ελληνικά'}]",Released,NaN,Afstiros katallilo,7.5,4.0
34919,NaN,0,[],43524,en,,0.004794,[],"[{'iso_3166_1': 'US', 'name': 'United States o...",1942-09-21,0.0,79.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Iceland,0.0,0.0


Atributo runtime

In [65]:
# Muestra el conteo de la duración de películas que más se repiten
movies_df['runtime'].value_counts(dropna= False)

,count
runtime,
90.0,2556
0.0,1558
100.0,1470
95.0,1412
93.0,1214
...,...
410.0,1
283.0,1
238.0,1


In [66]:
movies_df[movies_df['runtime'] == 0].status.value_counts()

,count
status,
Released,1496
Rumored,20
Post Production,13
In Production,7
Planned,4
Canceled,1


Examinación de los atributos con el formato JSON

In [67]:
movies_df.loc[0]

# Se encuentran 5 columnas en formato JSON: belongs_to_collection, genres, production_companies, production_countries, spoken_languages

,0
belongs_to_collection,"{'id': 10194, 'name': 'Toy Story Collection', ..."
budget,30000000
genres,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '..."
id,862
original_language,en
overview,"Led by Woody, Andy's toys live happily in his ..."
popularity,21.946943
production_companies,"[{'name': 'Pixar Animation Studios', 'id': 3}]"
production_countries,"[{'iso_3166_1': 'US', 'name': 'United States o..."
release_date,1995-10-30


In [68]:
# Valores de la primera fila de cada columna con formato JSON
print(f'Columna "belongs_to_collection": {movies_df.belongs_to_collection.loc[0]}')
print(f'Columna "genres": {movies_df.genres.loc[0]}')
print(f'Columna "production_companies": {movies_df.production_companies.loc[0]}')
print(f'Columna "production_countries": {movies_df.production_countries.loc[0]}')
print(f'Columna "spoken_languages": {movies_df.spoken_languages.loc[0]}')

Columna "belongs_to_collection": {'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}
Columna "genres": [{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]
Columna "production_companies": [{'name': 'Pixar Animation Studios', 'id': 3}]
Columna "production_countries": [{'iso_3166_1': 'US', 'name': 'United States of America'}]
Columna "spoken_languages": [{'iso_639_1': 'en', 'name': 'English'}]


Irregularidades de los datos en el formato JSON


Atributo belongs_to_collection

In [69]:
# Identificar las filas que contienen cadenas de texto vacías, eliminando los espacios en blanco al principio y al final de cada cadena
movies_df.loc[movies_df['belongs_to_collection'].str.strip() == '']

# Busca películas donde la columna 'belongs_to_collection' contiene el marcador '{}'
# Probablemente indica un formato de datos específico o un error
filtro_btc = [index for index, valor in enumerate(movies_df['belongs_to_collection']) if '{}' in str(valor)]
# Extrae las filas del DataFrame que cumplen con la condición
movies_df.iloc[filtro_btc]

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count


Atributo genres

In [70]:
# Identificar las filas que contienen cadenas de texto vacías, eliminando los espacios en blanco al principio y al final de cada cadena
movies_df.loc[movies_df['genres'].str.strip() == '']

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count


In [71]:
filtro_genres = movies_df[movies_df['genres'].str.contains('[]', regex = False)]

filtro_genres.genres.head()

,genres
55,[]
83,[]
126,[]
137,[]
390,[]


In [72]:
vacios_genres_data = filtro_genres.genres.shape[0]
print(f'Cantidad de registros vacios: {vacios_genres_data}')

Cantidad de registros vacios: 2442


Atributo production_companies

In [73]:
# Identificar las filas que contienen cadenas de texto vacías, eliminando los espacios en blanco al principio y al final de cada cadena
movies_df.loc[movies_df['production_companies'].str.strip() == '']

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count


In [74]:
# Filtrar las filas de la columna 'production_companies' que esten vacios ([])
filtro_production_companies = movies_df[movies_df['production_companies'].str.contains('[]', na= False, regex = False)]

# Muestras las primeras filas luego de ser filtrado
filtro_production_companies.production_companies.head()

,production_companies
50,[]
52,[]
57,[]
58,[]
83,[]


In [75]:
vacios_pc_data = filtro_production_companies.production_companies.shape[0]
print(f'Cantidad de registros vacios: {vacios_pc_data}')

Cantidad de registros vacios: 11875


Atributo production_countries

In [76]:
# Identificar las filas que contienen cadenas de texto vacías, eliminando los espacios en blanco al principio y al final de cada cadena
movies_df.loc[movies_df['production_countries'].str.strip() == '']

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count


In [77]:
# Filtrar las filas de la columna 'production_countries' que esten vacios ([])
filtro_countries = movies_df[movies_df['production_countries'].str.contains('[]', na= False, regex = False)]

# Muestras las primeras filas luego de ser filtrado
filtro_countries.production_countries.head()

,production_countries
50,[]
55,[]
83,[]
106,[]
107,[]


In [78]:
vacios_country_data = filtro_countries.production_countries.shape[0]
print(f'Cantidad de registros vacios: {vacios_country_data}')

Cantidad de registros vacios: 6282


Atributo spoken_languages

In [79]:
# Identificar las filas que contienen cadenas de texto vacías, eliminando los espacios en blanco al principio y al final de cada cadena
movies_df.loc[movies_df['spoken_languages'].str.strip() == '']

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count


In [80]:
# Filtrar las filas de la columna 'spoken_languages' que esten vacios ([])
filtro_sl = movies_df[movies_df['spoken_languages'].str.contains('[]', na= False, regex = False)]

# Muestras las primeras filas luego de ser filtrado
filtro_sl.spoken_languages.head()

,spoken_languages
50,[]
83,[]
107,[]
126,[]
137,[]


In [81]:
vacios_sl_data = filtro_sl.spoken_languages.shape[0]
print(f'Cantidad de registros vacios: {vacios_sl_data}')

Cantidad de registros vacios: 3829


Desanidar la data

In [82]:
# Copia del dataset para no modificar los datos originales
copia_movies = movies_df.copy()

In [83]:
def stringified(data):
    """
    Convierte una cadena que representa una estructura de datos de Python en esa estructura.
    Args:
        data (str): La cadena a evaluar.
    Returns:
        El objeto Python correspondiente, o np.nan si la evaluación falla.
    """
    try:
        # Intentamos convertir la cadena en un objeto Python utilizando ast.literal_eval
        return ast.literal_eval(data)
    except (ValueError, SyntaxError):
        # Si ocurre un error de valor o de sintaxis, devolvemos np.nan
        return np.nan

In [84]:
# columnas de tipo string
columns_movies = ['belongs_to_collection', 'genres', 'production_companies', 'production_countries', 'spoken_languages']


for column in columns_movies:
    for i in range(len(copia_movies)):
        copia_movies.at[i, column] = stringified(copia_movies.at[i, column])

copia_movies.head(3)

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,7.7,5415.0
1,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0
2,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,en,A family wedding reignites the ancient feud be...,11.7129,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0


In [85]:
# Atributo 'belongs_to_collection'

def obtener_nombre(datos):
    #Obtiene el nombre de un diccionario.
    #Args: datos en un diccionario.
    # Returns:El valor de la clave 'name' si existe y no está vacío, de lo contrario, retorna np.nan.
    if isinstance(datos, dict):  # Verifica si es un diccionario
        try:
            return datos['name'] if datos['name'] else np.nan
        except KeyError:
            return np.nan
    else:
        # Si no es un diccionario, retorna NaN directamente
        return np.nan

In [86]:
# Aplicar obtener_nombre para el atributo belongs_to_collection dado que sus registros son del tipo dict
copia_movies['collection'] = copia_movies['belongs_to_collection'].apply(lambda x: obtener_nombre(x))
copia_movies['collection'].head()

,collection
0,Toy Story Collection
1,NaN
2,Grumpy Old Men Collection
3,NaN
4,Father of the Bride Collection


In [87]:
copia_movies['collection'].value_counts(dropna = False).head()

,count
collection,
NaN,40975
The Bowery Boys,29
Totò Collection,27
James Bond Collection,26
Zatôichi: The Blind Swordsman,26


In [88]:
def get_name(data):
    """
    Extrae los nombres de una lista de diccionarios utilizando el método get.

    Args:
    data: Una lista de diccionarios.

    Returns:
    Una cadena con los nombres concatenados, o np.nan si no hay nombres.
    """

    if not isinstance(data, list):
        return np.nan

    nombres = [d.get('name', '') for d in data]
    return ', '.join(nombres) if nombres else np.nan

In [89]:
columnas_peliculas = ['genres', 'production_companies', 'production_countries']


for columna in columnas_peliculas:
    copia_movies[columna] = copia_movies[columna].apply(get_name)

# Aplicando a la columna 'spoken_languages' y creando una nueva columna
copia_movies['languages'] = copia_movies['spoken_languages'].apply(get_name)

In [90]:
# Verificar si hay datos anidados
copia_movies[['collection', 'genres', 'production_companies', 'production_countries', 'languages']].head()

,collection,genres,production_companies,production_countries,languages
0,Toy Story Collection,"Animation, Comedy, Family",Pixar Animation Studios,United States of America,English
1,NaN,"Adventure, Fantasy, Family","TriStar Pictures, Teitler Film, Interscope Com...",United States of America,"English, Français"
2,Grumpy Old Men Collection,"Romance, Comedy","Warner Bros., Lancaster Gate",United States of America,English
3,NaN,"Comedy, Drama, Romance",Twentieth Century Fox Film Corporation,United States of America,English
4,Father of the Bride Collection,Comedy,"Sandollar Productions, Touchstone Pictures",United States of America,English


Atributo 'genres'

In [91]:
# Frecuencia de repetición del atributo 'genres'
copia_movies['genres'].value_counts(dropna = False).head()

,count
genres,
Drama,5000
Comedy,3621
Documentary,2723
NaN,2442
"Drama, Romance",1301


In [92]:
datos_vacios = copia_movies['genres'].isnull().sum()
print(f'Contiene {datos_vacios} datos nulos en la columna "genres"')
print(f'Contiene {vacios_genres_data} datos con listas vacios')

Contiene 2442 datos nulos en la columna "genres"
Contiene 2442 datos con listas vacios


In [93]:
copia_movies[copia_movies['genres'] == '']

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,collection,languages


Atributo production_companies

In [94]:
# Muestra los primeros valores que más se repiten
copia_movies['production_companies'].value_counts(dropna = False).head()

,count
production_companies,
NaN,11881
Metro-Goldwyn-Mayer (MGM),742
Warner Bros.,540
Paramount Pictures,505
Twentieth Century Fox Film Corporation,439


In [95]:
datos_vacios = copia_movies['production_companies'].isnull().sum()
print(f'Contiene {datos_vacios} datos nulos en la columna "production_companies"')
print(f'Contiene {vacios_pc_data} datos con listas vacios')

Contiene 11881 datos nulos en la columna "production_companies"
Contiene 11875 datos con listas vacios


In [96]:
copia_movies[copia_movies['production_companies'] == '']

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,collection,languages


Atributo production_countries

In [97]:
# Muestra los primeros valores que más se repiten
copia_movies['production_countries'].value_counts(dropna = False).head()

,count
production_countries,
United States of America,17851
NaN,6288
United Kingdom,2238
France,1654
Japan,1356


In [98]:
datos_vacios = copia_movies['production_countries'].isnull().sum()
print(f'Contiene {datos_vacios} datos nulos en la columna "production_countries"')
print(f'Contiene {vacios_country_data} datos con listas vacios')

Contiene 6288 datos nulos en la columna "production_countries"
Contiene 6282 datos con listas vacios


In [99]:
copia_movies[copia_movies['production_countries'] == '']

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,collection,languages


Atributo languages

In [100]:
# Muestra los valores que más se repiten
copia_movies['languages'].value_counts(dropna = False)

,count
languages,
English,22395
NaN,3835
Français,1853
日本語,1289
Italiano,1218
...,...
"Français, Latin, , Português, English",1
"Español, עִבְרִית, English",1
"English, Íslenska, Pусский",1


In [101]:
datos_vacios = copia_movies['languages'].isnull().sum()
print(f'Contiene {datos_vacios} datos nulos en la columna "languages"')
print(f'Contiene {vacios_sl_data} datos con listas vacios')

Contiene 3835 datos nulos en la columna "languages"
Contiene 3829 datos con listas vacios


In [102]:
copia_movies[copia_movies['languages'] == ''].languages.head()

,languages
773,
4184,
5283,
7778,
7798,


In [103]:
# Filas que contienen cadenas de texto vacías, se elimina los espacios en blanco al principio y al final de cada cadena
print(copia_movies.loc[copia_movies['languages'].str.strip() == ''].languages.shape[0], 'filas vacias')

123 filas vacias


In [104]:
copia_movies.loc[copia_movies['languages'].str.strip() == '', 'languages'] = np.nan

In [105]:
# Verificar que ya no hay strings vacíos
copia_movies[copia_movies['languages'] == ''].languages

,languages


Corrección de los datos

In [106]:
copia_movies.dtypes

,0
belongs_to_collection,object
budget,object
genres,object
id,object
original_language,object
overview,object
popularity,object
production_companies,object
production_countries,object
release_date,object


In [107]:
copia_movies.loc[1]

,1
belongs_to_collection,NaN
budget,65000000
genres,"Adventure, Fantasy, Family"
id,8844
original_language,en
overview,When siblings Judy and Peter discover an encha...
popularity,17.015539
production_companies,"TriStar Pictures, Teitler Film, Interscope Com..."
production_countries,United States of America
release_date,1995-12-15


Atributo id -> de tipo object a int

In [108]:
copia_movies[copia_movies['id'].str.contains('-', regex= False)]
copia_movies.head()

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,collection,languages
0,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"Animation, Comedy, Family",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,Pixar Animation Studios,United States of America,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,7.7,5415.0,Toy Story Collection,English
1,NaN,65000000,"Adventure, Fantasy, Family",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"TriStar Pictures, Teitler Film, Interscope Com...",United States of America,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,NaN,"English, Français"
2,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"Romance, Comedy",15602,en,A family wedding reignites the ancient feud be...,11.7129,"Warner Bros., Lancaster Gate",United States of America,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,Grumpy Old Men Collection,English
3,NaN,16000000,"Comedy, Drama, Romance",31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,Twentieth Century Fox Film Corporation,United States of America,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,NaN,English
4,"{'id': 96871, 'name': 'Father of the Bride Col...",0,Comedy,11862,en,Just when George Banks has recovered from his ...,8.387519,"Sandollar Productions, Touchstone Pictures",United States of America,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,Father of the Bride Collection,English


In [109]:
copia_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   belongs_to_collection  4494 non-null   object 
 1   budget                 45466 non-null  object 
 2   genres                 43024 non-null  object 
 3   id                     45466 non-null  object 
 4   original_language      45455 non-null  object 
 5   overview               44512 non-null  object 
 6   popularity             45461 non-null  object 
 7   production_companies   33585 non-null  object 
 8   production_countries   39178 non-null  object 
 9   release_date           45379 non-null  object 
 10  revenue                45460 non-null  float64
 11  runtime                45203 non-null  float64
 12  spoken_languages       45460 non-null  object 
 13  status                 45379 non-null  object 
 14  tagline                20408 non-null  object 
 15  ti

In [110]:
# Crear una máscara booleana para las filas a eliminar
mask = copia_movies.index.isin([19730, 29503, 35587])

# Invertir la máscara para mantener las filas que queremos
copia_movies = copia_movies[~mask]

# Ver las modificaciones
copia_movies[copia_movies['id'].str.contains('-', regex = False)]

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,collection,languages


In [111]:
# Converción: columna 'id' de tipo de dato object a tipo de datos int32
copia_movies['id'] = copia_movies['id'].astype('int32')

# ver el tipo del dato luego de la modificación
copia_movies['id'].dtypes

dtype('int32')

In [112]:
copia_movies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45463 entries, 0 to 45465
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   belongs_to_collection  4491 non-null   object 
 1   budget                 45463 non-null  object 
 2   genres                 43021 non-null  object 
 3   id                     45463 non-null  int32  
 4   original_language      45452 non-null  object 
 5   overview               44509 non-null  object 
 6   popularity             45460 non-null  object 
 7   production_companies   33585 non-null  object 
 8   production_countries   39178 non-null  object 
 9   release_date           45376 non-null  object 
 10  revenue                45460 non-null  float64
 11  runtime                45203 non-null  float64
 12  spoken_languages       45460 non-null  object 
 13  status                 45379 non-null  object 
 14  tagline                20408 non-null  object 
 15  title  

Atributo budget -> de tipo Object a int

In [113]:
copia_movies[copia_movies['budget'].str.contains('-', regex= False)]
copia_movies.head()

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,collection,languages
0,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"Animation, Comedy, Family",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,Pixar Animation Studios,United States of America,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,7.7,5415.0,Toy Story Collection,English
1,NaN,65000000,"Adventure, Fantasy, Family",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"TriStar Pictures, Teitler Film, Interscope Com...",United States of America,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,NaN,"English, Français"
2,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"Romance, Comedy",15602,en,A family wedding reignites the ancient feud be...,11.7129,"Warner Bros., Lancaster Gate",United States of America,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,Grumpy Old Men Collection,English
3,NaN,16000000,"Comedy, Drama, Romance",31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,Twentieth Century Fox Film Corporation,United States of America,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,NaN,English
4,"{'id': 96871, 'name': 'Father of the Bride Col...",0,Comedy,11862,en,Just when George Banks has recovered from his ...,8.387519,"Sandollar Productions, Touchstone Pictures",United States of America,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,Father of the Bride Collection,English


In [114]:
copia_movies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45463 entries, 0 to 45465
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   belongs_to_collection  4491 non-null   object 
 1   budget                 45463 non-null  object 
 2   genres                 43021 non-null  object 
 3   id                     45463 non-null  int32  
 4   original_language      45452 non-null  object 
 5   overview               44509 non-null  object 
 6   popularity             45460 non-null  object 
 7   production_companies   33585 non-null  object 
 8   production_countries   39178 non-null  object 
 9   release_date           45376 non-null  object 
 10  revenue                45460 non-null  float64
 11  runtime                45203 non-null  float64
 12  spoken_languages       45460 non-null  object 
 13  status                 45379 non-null  object 
 14  tagline                20408 non-null  object 
 15  title  

In [115]:
# Filtrar filas que contengan el caracter ','
copia_movies[copia_movies['budget'].str.contains(',', regex = False)]

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,collection,languages


In [116]:
# Converción: columna 'budget' de tipo de dato object a tipo de datos int32
copia_movies['budget'] = copia_movies['budget'].astype('int32')

# ver el tipo del dato luego de la modificación
copia_movies['budget'].dtypes

dtype('int32')

In [117]:
copia_movies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45463 entries, 0 to 45465
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   belongs_to_collection  4491 non-null   object 
 1   budget                 45463 non-null  int32  
 2   genres                 43021 non-null  object 
 3   id                     45463 non-null  int32  
 4   original_language      45452 non-null  object 
 5   overview               44509 non-null  object 
 6   popularity             45460 non-null  object 
 7   production_companies   33585 non-null  object 
 8   production_countries   39178 non-null  object 
 9   release_date           45376 non-null  object 
 10  revenue                45460 non-null  float64
 11  runtime                45203 non-null  float64
 12  spoken_languages       45460 non-null  object 
 13  status                 45379 non-null  object 
 14  tagline                20408 non-null  object 
 15  title  

Atributo popularity -> de tipo object a float

In [118]:
# Filtramos las filas donde 'popularity' contiene un punto
filtro_popularidad = copia_movies[copia_movies['popularity'].str.contains('.', na= False, regex = False)]

# Mostramos las primeras 5 filas de la columna 'popularity' del DataFrame filtrado
filtro_popularidad.popularity.head()

,popularity
0,21.946943
1,17.015539
2,11.7129
3,3.859495
4,8.387519


In [119]:
copia_movies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45463 entries, 0 to 45465
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   belongs_to_collection  4491 non-null   object 
 1   budget                 45463 non-null  int32  
 2   genres                 43021 non-null  object 
 3   id                     45463 non-null  int32  
 4   original_language      45452 non-null  object 
 5   overview               44509 non-null  object 
 6   popularity             45460 non-null  object 
 7   production_companies   33585 non-null  object 
 8   production_countries   39178 non-null  object 
 9   release_date           45376 non-null  object 
 10  revenue                45460 non-null  float64
 11  runtime                45203 non-null  float64
 12  spoken_languages       45460 non-null  object 
 13  status                 45379 non-null  object 
 14  tagline                20408 non-null  object 
 15  title  

In [120]:
# Conversión: columna 'popularity' de tipo object a tipo de datos float
copia_movies['popularity'] = copia_movies['popularity'].astype('float64')

# ver el tipo del dato luego de la modificación
copia_movies['popularity'].dtypes

dtype('float64')

In [121]:
copia_movies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45463 entries, 0 to 45465
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   belongs_to_collection  4491 non-null   object 
 1   budget                 45463 non-null  int32  
 2   genres                 43021 non-null  object 
 3   id                     45463 non-null  int32  
 4   original_language      45452 non-null  object 
 5   overview               44509 non-null  object 
 6   popularity             45460 non-null  float64
 7   production_companies   33585 non-null  object 
 8   production_countries   39178 non-null  object 
 9   release_date           45376 non-null  object 
 10  revenue                45460 non-null  float64
 11  runtime                45203 non-null  float64
 12  spoken_languages       45460 non-null  object 
 13  status                 45379 non-null  object 
 14  tagline                20408 non-null  object 
 15  title  

Atributo release_date -> de tipo object a datetime

In [122]:
# Verificación del formato fecha
copia_movies['release_date'].head()

,release_date
0,1995-10-30
1,1995-12-15
2,1995-12-22
3,1995-12-22
4,1995-02-10


In [123]:
copia_movies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45463 entries, 0 to 45465
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   belongs_to_collection  4491 non-null   object 
 1   budget                 45463 non-null  int32  
 2   genres                 43021 non-null  object 
 3   id                     45463 non-null  int32  
 4   original_language      45452 non-null  object 
 5   overview               44509 non-null  object 
 6   popularity             45460 non-null  float64
 7   production_companies   33585 non-null  object 
 8   production_countries   39178 non-null  object 
 9   release_date           45376 non-null  object 
 10  revenue                45460 non-null  float64
 11  runtime                45203 non-null  float64
 12  spoken_languages       45460 non-null  object 
 13  status                 45379 non-null  object 
 14  tagline                20408 non-null  object 
 15  title  

In [124]:
# Conversión: columna 'release_date' de tipo object a tipo datetime64
copia_movies['release_date'] = pd.to_datetime(copia_movies['release_date'], format = '%Y-%m-%d')
copia_movies.release_date.dtypes

dtype('<M8[ns]')

In [125]:
copia_movies['release_date'].isna().sum()

87

In [126]:
copia_movies = copia_movies.dropna(subset=['release_date'])

In [127]:
copia_movies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45376 entries, 0 to 45465
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   belongs_to_collection  4488 non-null   object        
 1   budget                 45376 non-null  int32         
 2   genres                 42992 non-null  object        
 3   id                     45376 non-null  int32         
 4   original_language      45365 non-null  object        
 5   overview               44435 non-null  object        
 6   popularity             45376 non-null  float64       
 7   production_companies   33580 non-null  object        
 8   production_countries   39165 non-null  object        
 9   release_date           45376 non-null  datetime64[ns]
 10  revenue                45376 non-null  float64       
 11  runtime                45130 non-null  float64       
 12  spoken_languages       45376 non-null  object        
 13  status

In [128]:
# verificación.

# Obtener la fecha y hora actual
ahora = dt.datetime.now()
print(ahora)

2024-09-18 04:31:45.757397


In [129]:
# Convertir fecha y hora actual en un objeto (datatime64[ns])
fecha_hora_actual = np.datetime64('now')
fecha_hora_actual64 = pd.to_datetime(fecha_hora_actual)
copia_movies[copia_movies['release_date'] > fecha_hora_actual64]

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,collection,languages


In [130]:
# Verificar que los tipos de dato sean correctos
copia_movies.dtypes

,0
belongs_to_collection,object
budget,int32
genres,object
id,int32
original_language,object
overview,object
popularity,float64
production_companies,object
production_countries,object
release_date,datetime64[ns]


Eliminar datos duplicados

In [131]:
copia_movies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45376 entries, 0 to 45465
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   belongs_to_collection  4488 non-null   object        
 1   budget                 45376 non-null  int32         
 2   genres                 42992 non-null  object        
 3   id                     45376 non-null  int32         
 4   original_language      45365 non-null  object        
 5   overview               44435 non-null  object        
 6   popularity             45376 non-null  float64       
 7   production_companies   33580 non-null  object        
 8   production_countries   39165 non-null  object        
 9   release_date           45376 non-null  datetime64[ns]
 10  revenue                45376 non-null  float64       
 11  runtime                45130 non-null  float64       
 12  spoken_languages       45376 non-null  object        
 13  status

In [132]:
peliculas = copia_movies[['budget', 'genres',	'id',	'original_language', 'overview', 'popularity',
                        'production_companies', 'production_countries',	'release_date',	'revenue',
                        'runtime', 'status', 'tagline', 'title',	'vote_average',
                        'vote_count',	'collection',	'languages']].copy()

In [133]:
# Cantidad de duplicados en la columna 'id'
peliculas[peliculas.duplicated(subset = 'id', keep = False)].shape[0]

59

In [134]:
# Cuántas veces se repite cada valor en el atributo 'id' de forma descendente
frecuencia_id = peliculas.groupby('id').size()
frecuencia_id = frecuencia_id[frecuencia_id >= 2]
frecuencia_id = frecuencia_id.sort_values(ascending= False)
frecuencia_id.head()

,0
id,
141971,3
4912,2
84198,2
265189,2
168538,2


In [135]:
duplicados_id = [141971, 5511, 168538, 18440, 265189, 11115, 42495, 152795, 298721, 25541, 105045, 119916, 159849, 23305, 97995, 99080]
peliculas_duplicadas = peliculas.loc[peliculas['id'].isin(duplicados_id)].sort_values('id')
peliculas_duplicadas.head(4)

,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,status,tagline,title,vote_average,vote_count,collection,languages
7345,0,"Crime, Drama, Thriller",5511,fr,Hitman Jef Costello is a perfectionist who alw...,9.091288,"Fida cinematografica, Compagnie Industrielle e...","France, Italy",1967-10-25,39481.0,105.0,Released,There is no solitude greater than that of the ...,Le Samouraï,7.9,187.0,NaN,Français
9165,0,"Crime, Drama, Thriller",5511,fr,Hitman Jef Costello is a perfectionist who alw...,9.091288,"Fida cinematografica, Compagnie Industrielle e...","France, Italy",1967-10-25,39481.0,105.0,Released,There is no solitude greater than that of the ...,Le Samouraï,7.9,187.0,NaN,Français
24844,0,"Comedy, Drama",11115,en,As an ex-gambler teaches a hot-shot college ki...,6.880365,"Andertainment Group, Crescent City Pictures, T...",United States of America,2008-01-29,0.0,85.0,Released,NaN,Deal,5.2,22.0,NaN,English
14012,0,"Comedy, Drama",11115,en,As an ex-gambler teaches a hot-shot college ki...,6.880365,"Andertainment Group, Crescent City Pictures, T...",United States of America,2008-01-29,0.0,85.0,Released,NaN,Deal,5.2,22.0,NaN,English


In [136]:
peliculas.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45376 entries, 0 to 45465
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   budget                45376 non-null  int32         
 1   genres                42992 non-null  object        
 2   id                    45376 non-null  int32         
 3   original_language     45365 non-null  object        
 4   overview              44435 non-null  object        
 5   popularity            45376 non-null  float64       
 6   production_companies  33580 non-null  object        
 7   production_countries  39165 non-null  object        
 8   release_date          45376 non-null  datetime64[ns]
 9   revenue               45376 non-null  float64       
 10  runtime               45130 non-null  float64       
 11  status                45296 non-null  object        
 12  tagline               20394 non-null  object        
 13  title                

In [137]:
# Verificar diferencias
sustitucion = peliculas[peliculas['id'] == 20][['id', 'popularity']]
sustitucion

,id,popularity
6621,20,10.310508


In [138]:
peliculas.drop_duplicates(subset='id', keep='first', inplace=True)

In [139]:
if peliculas['id'].isin(id_duplicado).any():
    print('Aún hay valores duplicados')
else: print('Se eliminaron todos los valores duplicados')

Se eliminaron todos los valores duplicados


In [140]:
peliculas[peliculas['id'].isin(duplicados_id)].sort_values(by = 'id').head()

,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,status,tagline,title,vote_average,vote_count,collection,languages
7345,0,"Crime, Drama, Thriller",5511,fr,Hitman Jef Costello is a perfectionist who alw...,9.091288,"Fida cinematografica, Compagnie Industrielle e...","France, Italy",1967-10-25,39481.0,105.0,Released,There is no solitude greater than that of the ...,Le Samouraï,7.9,187.0,NaN,Français
14012,0,"Comedy, Drama",11115,en,As an ex-gambler teaches a hot-shot college ki...,6.880365,"Andertainment Group, Crescent City Pictures, T...",United States of America,2008-01-29,0.0,85.0,Released,NaN,Deal,5.2,22.0,NaN,English
14000,0,"Action, Horror, Science Fiction",18440,en,When a comet strikes Earth and kicks up a clou...,1.436085,NaN,United States of America,2007-01-01,0.0,89.0,Released,NaN,Days of Darkness,5.0,5.0,NaN,English
8068,0,"Adventure, Animation, Drama, Action, Foreign",23305,en,"In feudal India, a warrior (Khan) who renounce...",1.967992,Filmfour,"France, Germany, India, United Kingdom",2001-09-23,0.0,86.0,Released,NaN,The Warrior,6.3,15.0,NaN,हिन्दी
17229,0,Drama,25541,da,Former Danish servicemen Lars and Jimmy are th...,2.587911,NaN,"Sweden, Denmark",2009-10-21,0.0,90.0,Released,NaN,Brotherhood,7.1,21.0,NaN,Dansk


In [141]:
peliculas.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45346 entries, 0 to 45465
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   budget                45346 non-null  int32         
 1   genres                42962 non-null  object        
 2   id                    45346 non-null  int32         
 3   original_language     45335 non-null  object        
 4   overview              44405 non-null  object        
 5   popularity            45346 non-null  float64       
 6   production_companies  33557 non-null  object        
 7   production_countries  39138 non-null  object        
 8   release_date          45346 non-null  datetime64[ns]
 9   revenue               45346 non-null  float64       
 10  runtime               45100 non-null  float64       
 11  status                45266 non-null  object        
 12  tagline               20383 non-null  object        
 13  title                

In [142]:
peliculas[peliculas.duplicated(subset= 'id', keep= False)].shape[0]

0

In [143]:
peliculas.head()

,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,status,tagline,title,vote_average,vote_count,collection,languages
0,30000000,"Animation, Comedy, Family",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,Pixar Animation Studios,United States of America,1995-10-30,373554033.0,81.0,Released,NaN,Toy Story,7.7,5415.0,Toy Story Collection,English
1,65000000,"Adventure, Fantasy, Family",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"TriStar Pictures, Teitler Film, Interscope Com...",United States of America,1995-12-15,262797249.0,104.0,Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,NaN,"English, Français"
2,0,"Romance, Comedy",15602,en,A family wedding reignites the ancient feud be...,11.712900,"Warner Bros., Lancaster Gate",United States of America,1995-12-22,0.0,101.0,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,Grumpy Old Men Collection,English
3,16000000,"Comedy, Drama, Romance",31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,Twentieth Century Fox Film Corporation,United States of America,1995-12-22,81452156.0,127.0,Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,NaN,English
4,0,Comedy,11862,en,Just when George Banks has recovered from his ...,8.387519,"Sandollar Productions, Touchstone Pictures",United States of America,1995-02-10,76578911.0,106.0,Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,Father of the Bride Collection,English


In [144]:
peliculas.shape

(45346, 18)

In [145]:
peliculas.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45346 entries, 0 to 45465
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   budget                45346 non-null  int32         
 1   genres                42962 non-null  object        
 2   id                    45346 non-null  int32         
 3   original_language     45335 non-null  object        
 4   overview              44405 non-null  object        
 5   popularity            45346 non-null  float64       
 6   production_companies  33557 non-null  object        
 7   production_countries  39138 non-null  object        
 8   release_date          45346 non-null  datetime64[ns]
 9   revenue               45346 non-null  float64       
 10  runtime               45100 non-null  float64       
 11  status                45266 non-null  object        
 12  tagline               20383 non-null  object        
 13  title                

Unión de los datasets

Verificar similitudes por el atributo 'id'

In [146]:
# Imprimir la cantidad de id
print('Cantidad de id de credicts: ', creditos.id.shape[0])
print('Cantidad de id movies: ', peliculas.id.shape[0])

Cantidad de id de credicts:  45432
Cantidad de id movies:  45346


In [147]:
print('Id duplicados en credicts: ', peliculas[peliculas.duplicated(subset = 'id', keep = False)].shape[0])
print('Id duplicados en movies: ', peliculas[peliculas.duplicated(subset = 'id', keep = False)].shape[0])

Id duplicados en credicts:  0
Id duplicados en movies:  0


In [148]:
# Seleccionar el atributo 'id' del df 'peliculas' y se crea un nuevo df para hacer el test ('peliculas_test')
peliculas_test = peliculas[['id']]

# Seleccionar el atributo 'id' del df 'creditos' y se crea un nuevo df para hacer el test ('creditos_test')
creditos_test = creditos[['id']]

# Unir los nuevos df con 'merge'
comparacion_id = peliculas_test.merge(creditos_test, how='outer', indicator= 'union')
list_both = np.array([comparacion_id.union == 'both']).sum()
list_left_only = np.array([comparacion_id.union == 'left_only']).sum()
list_right_only = np.array([comparacion_id.union == 'right_only']).sum()


In [149]:
# Se imprimen los resultados de la unión
print(f'Cantidad de Id presentes en ambos: {list_both}')
print(f'Cantidad de Id que estan presentes en movies: {list_left_only}')
print(f'Cantidad de Id que estan presentes en credits: {list_right_only}')

Cantidad de Id presentes en ambos: 45345
Cantidad de Id que estan presentes en movies: 1
Cantidad de Id que estan presentes en credits: 87


In [150]:
# Id que está presente sólo en "peliculas_test"
id_peliculas_only = comparacion_id.loc[comparacion_id['union'] == 'left_only', 'id']
print(f'Id que está presente solo en peliculas: {id_peliculas_only}')

Id que está presente solo en peliculas: 42783    401840
Name: id, dtype: int64


In [151]:
# Verificar si el id está en el df de creditos
credits_csv[credits_csv['id']== 401840]

,cast,crew,id


In [152]:
# Verificar que la película corresponde el id = 401840
peliculas[peliculas['id'] == 401840][['id', 'title', 'popularity', 'release_date']]

,id,title,popularity,release_date
42883,401840,School's out,0.207775,2017-05-30


In [153]:
union_df = peliculas.merge(creditos, on = 'id', how = 'outer')
union_df.head()

,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,status,tagline,title,vote_average,vote_count,collection,languages,director,actores
0,30000000.0,"Animation, Comedy, Family",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,Pixar Animation Studios,United States of America,1995-10-30,373554033.0,81.0,Released,NaN,Toy Story,7.7,5415.0,Toy Story Collection,English,John Lasseter,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney,..."
1,65000000.0,"Adventure, Fantasy, Family",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"TriStar Pictures, Teitler Film, Interscope Com...",United States of America,1995-12-15,262797249.0,104.0,Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,NaN,"English, Français",Joe Johnston,"Robin Williams, Jonathan Hyde, Kirsten Dunst, ..."
2,0.0,"Romance, Comedy",15602,en,A family wedding reignites the ancient feud be...,11.712900,"Warner Bros., Lancaster Gate",United States of America,1995-12-22,0.0,101.0,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,Grumpy Old Men Collection,English,Howard Deutch,"Walter Matthau, Jack Lemmon, Ann-Margret, Soph..."
3,16000000.0,"Comedy, Drama, Romance",31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,Twentieth Century Fox Film Corporation,United States of America,1995-12-22,81452156.0,127.0,Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,NaN,English,Forest Whitaker,"Whitney Houston, Angela Bassett, Loretta Devin..."
4,0.0,Comedy,11862,en,Just when George Banks has recovered from his ...,8.387519,"Sandollar Productions, Touchstone Pictures",United States of America,1995-02-10,76578911.0,106.0,Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,Father of the Bride Collection,English,Charles Shyer,"Steve Martin, Diane Keaton, Martin Short, Kimb..."


In [154]:
union_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45433 entries, 0 to 45432
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   budget                45346 non-null  float64       
 1   genres                42962 non-null  object        
 2   id                    45433 non-null  int64         
 3   original_language     45335 non-null  object        
 4   overview              44405 non-null  object        
 5   popularity            45346 non-null  float64       
 6   production_companies  33557 non-null  object        
 7   production_countries  39138 non-null  object        
 8   release_date          45346 non-null  datetime64[ns]
 9   revenue               45346 non-null  float64       
 10  runtime               45100 non-null  float64       
 11  status                45266 non-null  object        
 12  tagline               20383 non-null  object        
 13  title           

In [155]:
entrada_original = union_df.shape[0]
union_df.shape

(45433, 20)

Registros que faltan

In [156]:
missing_values_count = union_df.isnull().sum()
columns = union_df.shape[1]
missing_values_count[0:columnas]

,0
budget,87
genres,2471
id,0
original_language,98
overview,1028
popularity,87
production_companies,11876
production_countries,6295
release_date,87
revenue,87


In [157]:
union_df = union_df.replace('', np.nan)

Eliminar el atributo Collection

Representa la mayor parte de valores nulos, no aporta nada relevante

In [158]:
union_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45433 entries, 0 to 45432
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   budget                45346 non-null  float64       
 1   genres                42962 non-null  object        
 2   id                    45433 non-null  int64         
 3   original_language     45335 non-null  object        
 4   overview              44405 non-null  object        
 5   popularity            45346 non-null  float64       
 6   production_companies  33557 non-null  object        
 7   production_countries  39138 non-null  object        
 8   release_date          45346 non-null  datetime64[ns]
 9   revenue               45346 non-null  float64       
 10  runtime               45100 non-null  float64       
 11  status                45266 non-null  object        
 12  tagline               20383 non-null  object        
 13  title           

In [159]:
union_df = union_df.drop(['collection'], axis = 1)

In [160]:
union_df.columns

Index(['budget', 'genres', 'id', 'original_language', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'languages', 'director', 'actores'],
      dtype='object')

Atributo release_date

In [161]:
date_nulos = union_df[union_df['release_date'].isnull()]
date_nulos.head(3)

,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,status,tagline,title,vote_average,vote_count,languages,director,actores
45346,NaN,NaN,36663,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
45347,NaN,NaN,38061,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Paul Cowan,None
45348,NaN,NaN,47934,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Roland Emmerich,None


In [162]:
union_df = union_df.replace('', np.nan)

In [163]:
# Imprimir la cantidad de valores nulos en los atributos "director" y "actors"
print('De las 87 filas con datos nulos en release_date:')
print('El atributo director tiene:', date_nulos['director'].isnull().sum(), 'datos nulos')
print('El atributo actors tiene:', date_nulos['actores'].isnull().sum(), 'datos nulos')
print('El atributo title tiene:', date_nulos['title'].isnull().sum(), 'datos nulos')
print('El atributo overview tiene:', date_nulos['overview'].isnull().sum(), 'datos nulos')

De las 87 filas con datos nulos en release_date:
El atributo director tiene: 52 datos nulos
El atributo actors tiene: 66 datos nulos
El atributo title tiene: 87 datos nulos
El atributo overview tiene: 87 datos nulos


In [164]:
date_nulos[date_nulos['title'].isnull()]

,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,status,tagline,title,vote_average,vote_count,languages,director,actores
45346,NaN,NaN,36663,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
45347,NaN,NaN,38061,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Paul Cowan,None
45348,NaN,NaN,47934,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Roland Emmerich,None
45349,NaN,NaN,76162,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
45350,NaN,NaN,82663,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Lawrence Gordon Clark,"Rob Lowe, Kenneth Cranham, Deborah Moore, Hann..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45428,NaN,NaN,448290,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
45429,NaN,NaN,449131,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Konstantin Murzenko,"Evgeniy Stychkin, Denis Burgazliev, Aleksandra..."
45430,NaN,NaN,452922,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,John Shrapnel
45431,NaN,NaN,455027,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None


In [165]:
# Eliminación de las filas donde la columna "release_date" tiene valores nulos
union_df = union_df.dropna(subset = ['release_date'])

In [166]:
# Cálculo de la cantidad de valores faltantes considerando los cambios en el atributo "release_date"
conteo_valores_faltantes = union_df.isnull().sum()

# % de datos faltantes respecto al total de los datos
total_cells = np.product(union_df.shape)
total_missing = conteo_valores_faltantes.sum()
percent_missing = (total_missing / total_cells) * 100
print(f'{round(percent_missing, 2)}%')

6.23%


Atributo overview

In [167]:
# Cálculo de datos faltantes en el atributo "overview" respecto al total de sus registros, medido en %
total_overview = union_df['overview'].shape[0]
total_missing_overview = union_df['overview'].isnull().sum()
percent_missing_overview = (total_missing_overview / total_overview) * 100
value = round(percent_missing_overview, 2)
print(f'De {total_overview} registros que posee la columna overview, {total_missing_overview} corresponden a los datos faltantes. Lo que representa el {value}% de los datos de la columna')

De 45346 registros que posee la columna overview, 941 corresponden a los datos faltantes. Lo que representa el 2.08% de los datos de la columna


In [168]:
# Mostrar un conteo de las descripciones que más se repiten en el atributo "overview"
union_df['overview'].value_counts(dropna = False).head()

,count
overview,
NaN,941
No overview found.,133
No Overview,7
,5
No movie overview available.,3


In [169]:
# Limpiar los valores faltantes de la columna 'overview'
union_df['overview'] = union_df['overview'].apply(lambda x: str(x).strip())
union_df['overview'] = union_df['overview'].apply(lambda x: re.sub(r'[^\w\s]','',x))
union_df['overview'] = union_df['overview'].apply(lambda x: x.lower())

# Reemplazar los valores faltantes con 'No overview found' o NaN
union_df['overview'] = union_df['overview'].apply(lambda x: 'no overview found' if x in ['nan', 'no overview', 'no overview found', 'no movie overview available', None] else x)
union_df['overview'] = union_df['overview'].apply(lambda x: np.nan if x == '' else x)

In [170]:
# Verificando que los cambios se realizaron
union_df['overview'].value_counts(dropna = False).head(4)

,count
overview,
no overview found,1087
NaN,6
adaptation of the jane austen novel,3
no overview yet,3


In [171]:
# Verificando si quedaron datos nulos
union_df['overview'].isnull().sum()

6

In [172]:
# Indice del valor nulo en la columna "overview"
null_index = union_df["overview"].isnull().idxmax()
# Imprime el valor de la columna "overview" para la fila correspondiente al índice "null_index"
union_df.loc[null_index, "overview"]

nan

In [173]:
union_df['overview'].fillna('no overview found', inplace=True)

In [174]:
# Cálculo del total de datos faltantes considerando los cambios en el atributo "overview"
conteo_valores_faltantes = union_df.isnull().sum()

# % de datos faltantes respecto al total de los datos en "union_df"
total_cells = np.product(union_df.shape)
total_missing = conteo_valores_faltantes.sum()
percent_missing = (total_missing / total_cells) * 100
print(f'{round(percent_missing, 2)}%')

6.12%


Atributo genres

In [175]:
# Cálculo de datos faltantes en el atributo "genres" respecto al total de sus registros, medido en %
total_genres = union_df['genres'].shape[0]
total_missing_genres = union_df['genres'].isnull().sum()
percent_missing_genres = (total_missing_genres / total_genres) * 100
value = round(percent_missing_genres, 2)
print(f'De {total_genres} registros que posee la columna genres, {total_missing_genres} corresponden a los datos faltantes. Lo que representa el {value}% de los datos de la columna')

De 45346 registros que posee la columna genres, 2384 corresponden a los datos faltantes. Lo que representa el 5.26% de los datos de la columna


In [176]:
# Mostrar un conteo de los géneros que más se repiten en el atributo "genres"
union_df['genres'].value_counts(dropna = False).head()

,count
genres,
Drama,4994
Comedy,3620
Documentary,2711
NaN,2384
"Drama, Romance",1300


In [177]:
# Identificar si retornaron filas que contienen cadenas de texto vacías
union_df.loc[union_df['genres'].str.strip() == ''].genres.shape[0]

0

In [178]:
# Reemplazar los valores de cadena vacía en la columna "genres" con valores NaN
union_df['genres'] = union_df['genres'].replace('', np.nan)

In [179]:
# Verificar cuantos registros nulos no se pudieron imputar
union_df['genres'].isnull().sum()

2384

In [180]:
# Función que verifica si hay géneros duplicados dentro de cada registro en la columna "genres"
union_df['genres'] = union_df['genres'].apply(lambda x: x.replace(' ', '').strip() if not pd.isnull(x) else x)

In [181]:
def has_duplicates(genres):
    if pd.isnull(genres):
        return False
    return len(set(genres.split(','))) < len(genres.split(','))

# Aplicar la función a la columna 'genres' y almacenar los resultados
duplicates = union_df['genres'].apply(has_duplicates)

# Imprimir los registros con duplicados
if duplicates.any():
    print('Los siguientes registros contienen géneros duplicados en la columna "genres":')
    for index, row in union_df[duplicates].iterrows():
        print(f'Índice: {index}, Géneros: {row["genres"]}')
else:
    print('No hay géneros duplicados en la columna "genres"')

No hay géneros duplicados en la columna "genres"


In [182]:
union_df['genres'] = union_df['genres'].str.lower()

In [183]:
# Copia de "union_df" para no afectar los registros nulos
copia_union = union_df.copy()

In [184]:
# Eliminar filas con valores nulos en 'genres'
copia_union = copia_union.dropna(subset=['genres'])

# Obtener una serie con la cuenta de cada género
genre_counts = copia_union['genres'].str.split(',').explode().value_counts()

# Obtener la lista de géneros únicos
genres_list = genre_counts.index.tolist()

# Imprimir los resultados
print(f'Lista de géneros presentes en la columna "genres":\n{genres_list}\n')
print(f'Cantidad de géneros únicos: {len(genres_list)}')

Lista de géneros presentes en la columna "genres":
['drama', 'comedy', 'thriller', 'romance', 'action', 'horror', 'crime', 'documentary', 'adventure', 'sciencefiction', 'family', 'mystery', 'fantasy', 'animation', 'foreign', 'music', 'history', 'war', 'western', 'tvmovie']

Cantidad de géneros únicos: 20


Atributo runtime

In [185]:
# Mostrar cuántas veces se repiten en el atributo "runtime" (tiempo en minutos)
union_df.runtime.value_counts().head()

,count
runtime,
90.0,2548
0.0,1535
100.0,1470
95.0,1409
93.0,1212


In [186]:
# Conteo de la cantidad de datos nulos en el atributo "runtime"
union_df.runtime.isnull().sum()

246

Atributo languages

In [187]:
# Mostrar un conteo de las tiempos en minutos que más se repiten en el atributo "languages"
union_df.languages.value_counts(dropna = False).head(5)

,count
languages,
English,22366
NaN,3889
Français,1850
日本語,1287
Italiano,1217


In [188]:
# Separar los lenguajes en múltiples filas utilizando el método explode()
filtro_languages = union_df.assign(languages = union_df['languages'].str.split(',')).explode('languages')

# Obtener el conteo de valores de cada lenguaje único
conteo_languages = filtro_languages['languages'].str.strip().value_counts(dropna = False)

In [189]:
# Mostrar el conteo de valores de cada lenguaje único
conteo_languages.sample(10)

,count
languages,
Eesti,50
Esperanto,7
ქართული,33
ภาษาไทย,175
ελληνικά,213
پښتو,8
No Language,319
Norsk,170
isiZulu,18


Irregularidad en atributos: title y overview

In [190]:
# Copia para no modificar todos los cambios realizados
merged = union_df.copy()

atributo title

In [191]:
# Identificar las filas que contienen cadenas de texto vacías, eliminando los espacios en blanco al principio y al final de cada cadena
merged.loc[merged['title'].str.strip() == '']

,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,status,tagline,title,vote_average,vote_count,languages,director,actores


In [192]:
# Verificar si hay caracteres especiales que no sean letras o números en la columna 'title'
special_chars_regex = re.compile('[^A-Za-z0-9\s]+')
contains_special_chars = merged['title'].str.contains(special_chars_regex)

# Mostrar las filas del DataFrame que contienen caracteres especiales
df_contains_special_chars = merged[contains_special_chars]
df_contains_special_chars_sorted = df_contains_special_chars.sort_values('title')
df_contains_special_chars_sorted.title.head()

,title
18735,!Women Art Revolution
30905,#1 Cheerleader Camp
36082,#Horror
23459,#chicagoGirl
27993,"$1,000 on the Black"


In [193]:
# Filtrar las filas donde el título de la película contiene un signo de dólar
dollar_titles = merged.loc[merged['title'].str.contains('\$')]
filas = dollar_titles.shape[0]
print(f'Hay {filas} filas que contienen el caracter especial $\n')
dollar_titles.title.head()

Hay 18 filas que contienen el caracter especial $



,title
5051,How to Beat the High Co$t of Living
7609,The First $20 Million Is Always the Hardest
9500,What the #$*! Do We (K)now!?
11860,Who the #$&% Is Jackson Pollock?
14820,$9.99


In [194]:
# Filtrar las filas donde el título de la película contiene un signo del hashtag
hashtag_titles = merged.loc[merged['title'].str.contains('\#')]
filas = hashtag_titles.shape[0]
print(f'Hay {filas} filas que contienen el caracter especial #\n')
hashtag_titles.title.head()

Hay 12 filas que contienen el caracter especial #



,title
5727,Revolution #9
9217,Female Prisoner #701: Scorpion
9500,What the #$*! Do We (K)now!?
11860,Who the #$&% Is Jackson Pollock?
12731,Gunnin' for That #1 Spot


In [195]:
# Corregir el título de algunas películas
merged.loc[merged['title'] == 'What the #$*! Do We (K)now!?', 'title'] = 'What the Bleep Do We Know!?'
merged.loc[merged['title'] == 'Night #1', 'title'] = 'Nuit #1'
merged.loc[merged['title'] == "Richard Pryor: I Ain't Dead Yet, #*%$#@!!", 'title'] = "Richard Pryor: I Ain't Dead Yet"
merged.loc[merged['title'] == 'How to Beat the High Co$t of Living', 'title'] = 'How to Beat the High Cost of Living'

Atributo overview

In [196]:
# Patrón de búsqueda para caracteres especiales no numéricos
pattern = r'[^a-zA-Z0-9\s]'
# Crear una máscara booleana que indica si cada descripción de película contiene caracteres especiales no numéricos
mask = merged['overview'].str.contains(pattern, regex=True)
# Filtrar el dataframe para incluir solo las filas que contienen caracteres especiales no numéricos
merged_con_caracteres = merged[mask]
# Imprimir el dataframe resultante
merged_con_caracteres['overview']

,overview
2,a family wedding reignites the ancient feud be...
45,soontobewed graduate student finn dodd develop...
72,in france during world war ii a poor and illit...
76,a look into the many lives of christa päffgen ...
139,a gay cabaret owner and his drag queen compani...
...,...
45231,in an era in which it is difficult for the tru...
45279,the tuner russian настройщик nastroyshchik is ...
45290,somewhere deep deep in russia there is a town ...
45293,у девушки даши приехавшей с подругой покорять ...


In [197]:
# Se crea una copia de la columna "overview" para aplicar los cambios en "overview_clean"
merged['overview_clean'] = merged['overview'].copy()

In [198]:
# Patrón de búsqueda para caracteres especiales no numéricos
pattern = r'[^a-zA-Z0-9\s]'
# Crear una máscara booleana que indica si cada descripción de película contiene caracteres especiales no numéricos
mask = merged['overview_clean'].str.contains(pattern, regex=True)
# Filtrar el dataframe para incluir solo las filas que contienen caracteres especiales no numéricos
merged_con_caracteres = merged[mask]
# Imprimir el dataframe resultante
merged_con_caracteres['overview_clean']

,overview_clean
2,a family wedding reignites the ancient feud be...
45,soontobewed graduate student finn dodd develop...
72,in france during world war ii a poor and illit...
76,a look into the many lives of christa päffgen ...
139,a gay cabaret owner and his drag queen compani...
...,...
45231,in an era in which it is difficult for the tru...
45279,the tuner russian настройщик nastroyshchik is ...
45290,somewhere deep deep in russia there is a town ...
45293,у девушки даши приехавшей с подругой покорять ...


In [199]:
# Filtrar las filas que no contengan "English"
filtered_df = merged_con_caracteres[~merged_con_caracteres['languages'].str.contains('English', na = False)]
filtered_df[['id', 'overview_clean', 'languages']]

,id,overview_clean,languages
72,48750,in france during world war ii a poor and illit...,Français
151,649,beautiful young housewife séverine serizy cann...,"Français, , Español"
196,5967,this simple romantic tragedy begins in 1957 gu...,Français
302,110,red this is the third film from the trilogy by...,Français
317,12527,havana cuba 1979 flamboyantly gay artist diego...,Español
...,...,...,...
45231,63990,in an era in which it is difficult for the tru...,"Français, Deutsch, Polski"
45279,61385,the tuner russian настройщик nastroyshchik is ...,Pусский
45290,100152,somewhere deep deep in russia there is a town ...,NaN
45293,63281,у девушки даши приехавшей с подругой покорять ...,Pусский


In [200]:
# Crear un objeto WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [201]:
# Función para limpiar las descripciones de películas
def limpiar_descripcion(descripcion):
  # Convertir toda la descripción a minúsculas
  descripcion = descripcion.lower()
   # Eliminar los caracteres especiales de la descripción utilizando una expresión regular
  descripcion = re.sub(r'[^a-zA-Z0-9\s]', '', descripcion)
  # Tokenizar la descripción en palabras individuales
  palabras = word_tokenize(descripcion)
  # Lematizar cada palabra de la descripción para obtener su forma base (lemmatization)
  palabras_lemmatizadas = [lemmatizer.lemmatize(palabra) for palabra in palabras]
  # Unir las palabras lematizadas en una sola cadena de texto nuevamente
  descripcion_limpia = ' '.join(palabras_lemmatizadas)
  # Devolver la descripción limpia
  return descripcion_limpia

# Aplicar la función 'limpiar_descripcion' a la nueva columna 'overview_clean'
merged['overview_clean'] = merged['overview_clean'].apply(limpiar_descripcion)

In [202]:
# Patrón de búsqueda para caracteres especiales no numéricos
pattern = r'[^a-zA-Z0-9\s]'
# Crear una máscara booleana que indica si cada descripción de película contiene caracteres especiales no numéricos
mask = merged['overview_clean'].str.contains(pattern, regex=True)
# Filtrar el dataframe para incluir solo las filas que contienen caracteres especiales no numéricos
merged_con_caracteres = merged[mask]
# Imprimir el dataframe resultante
merged_con_caracteres['overview_clean']

,overview_clean


In [203]:
# Se verifica que en el nuevo atributo no hayan datos con strings vacios
merged[merged['overview_clean'] == '']

,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,status,tagline,title,vote_average,vote_count,languages,director,actores,overview_clean
29988,0.0,drama,64268,ru,ульяна тулина необычный человек во взрослом т...,0.803834,NaN,Russia,2005-07-21,0.0,95.0,Released,NaN,Dura,4.3,3.0,Pусский,Maxim Korostyshevsky,"Oksana Korostyshevskaya, Aleksandr Baluev, Reg...",
34180,0.0,comedy,68192,en,фильм создан в жанре музыкальной комедии по мо...,0.035723,NaN,NaN,1978-06-06,0.0,145.0,Released,NaN,Cabbages and Kings,7.0,1.0,NaN,Nikolai Rasheyev,"Valentin Gaft, Nikolai Karachentsov, Ernst Rom...",
34191,0.0,"drama,war",94696,ru,по вызову своего жениха светлана поехала в зах...,0.099098,NaN,NaN,1962-06-01,0.0,106.0,Released,NaN,Four Winds of Heaven,5.0,2.0,Pусский,Stanislav Rostotsky,"Larisa Luzhina, Vyacheslav Tikhonov, Klara Luc...",
42162,0.0,comedy,87908,el,ο μπάμπης βλαδίμηρος κυριακίδης ο άγης δημήτρη...,0.271513,NaN,Greece,2011-12-22,0.0,109.0,Released,The treasure hunt,The Island 2: The Hunt for the Lost Treasure,6.3,6.0,ελληνικά,Antonis Aggelopoulos,"Vladimiros Kiriakidis, Mihalis Marinos, Dimitr...",
43052,3500000.0,action,56372,ru,артем колчин был одним из многих но он хотел с...,0.768181,NaN,Russia,2005-03-05,8262833.0,126.0,Released,NaN,Shadowboxing,5.0,13.0,Pусский,Aleksei Sidorov,"Andrei Panin, Elena Panova, Denis Nikiforov, D...",
43429,2000000.0,NaN,31059,en,рафшан и джумжуд гастарбайтеры из нубарашена н...,0.995285,NaN,Russia,2010-01-21,22212223.0,85.0,Released,NaN,Nasha Russia: Yaytsa sudby,4.6,15.0,Pусский,Gleb Orlov,"Sergey Svetlakov, Mikhail Galustyan, Viktor Ve...",
43999,0.0,"comedy,drama",67545,ru,андрей простой русский оперуполномоченный он ...,0.082394,NaN,NaN,1992-01-01,0.0,109.0,Released,NaN,Patrioticheskaya Komediya,4.0,2.0,NaN,Vladimir Khotinenko,"Sergei Makovetsky, Aleksey Serebryakov, Larisa...",
44049,0.0,"action,crime",78323,ru,москва наши дни в центре города из машины двое...,0.939805,NaN,Russia,2011-10-13,2252547.0,89.0,Released,NaN,Bablo,6.8,8.0,Pусский,Konstantin Buslov,"Мария Берсенева, Roman Madyanov, Gia Gogishvil...",
44747,0.0,NaN,56304,ru,главный герой александр крупный бизнесмен и вл...,0.468259,NaN,Russia,2011-01-20,0.0,93.0,Released,NaN,Satisfaktsiya,5.6,4.0,Pусский,Anna Matison,"Denis Burgazliev, Evgeniy Grishkovets",
45293,2000000.0,"romance,drama",63281,en,у девушки даши приехавшей с подругой покорять ...,0.121844,Profit,Russia,2010-09-30,1268793.0,107.0,Released,NaN,Pro Lyuboff,4.0,3.0,Pусский,Olga Subbotina,"Fyodor Bondarchuk, Oksana Fandera, Juozas Budr...",


Atributos revenue y budget

In [204]:
# Conteo de la cantidad de valores nulos en la columna "revenue"
merged['revenue'].isnull().sum()

0

In [205]:
# Conteno de la cantidad de valores nulos en la columna "budget"
merged['budget'].isnull().sum()

0

Atributo release_year

In [206]:
# Se agrega una nueva columna llamada "release_year" al DataFrame "merged" que contiene el año de lanzamiento de cada película.
merged['release_year'] = merged['release_date'].dt.year.astype('int32')
merged['release_year'].head()

,release_year
0,1995
1,1995
2,1995
3,1995
4,1995


Atributo return

In [207]:
# Conteo de la cantidad de registros con valor igual a 0 en el atributo "budget"
np.array([merged['budget'] == 0]).sum()

36470

In [208]:
# Conteo de la cantidad de registros con valor igual a 0 en el atributo "revenue"
np.array([merged['revenue'] == 0]).sum()

37949

In [209]:
# Se define una función para calcular el retorno de inversión para cada película en "merged".
def calculate_return(row):
  # Si el valor en la columna "budget" es igual a cero, se devuelve 0 como retorno de inversión.
  if row['budget'] == 0:
    return 0
  else:
    # Se calcula el retorno de inversión dividiendo los valores entre las columnas "revenue" y "budget".
    return row['revenue'] / row['budget']

In [210]:
# Se crea la columna 'return' utilizando la función 'calculate_return'
merged['return'] = merged.apply(calculate_return, axis = 1)
# Se reemplazan los valores faltantes con 0
merged['return'] = merged['return'].fillna(0)
# Se imprime los primeros 5 valores del atributo "return"
merged['return'].head()

,return
0,12.451801
1,4.043035
2,0.000000
3,5.090760
4,0.000000


# Carga

In [211]:
# Imprimir las primeras 5 filas del Dataframe "merged"
merged.head()

,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,...,tagline,title,vote_average,vote_count,languages,director,actores,overview_clean,release_year,return
0,30000000.0,"animation,comedy,family",862,en,led by woody andys toys live happily in his ro...,21.946943,Pixar Animation Studios,United States of America,1995-10-30,373554033.0,...,NaN,Toy Story,7.7,5415.0,English,John Lasseter,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney,...",led by woody andys toy live happily in his roo...,1995,12.451801
1,65000000.0,"adventure,fantasy,family",8844,en,when siblings judy and peter discover an encha...,17.015539,"TriStar Pictures, Teitler Film, Interscope Com...",United States of America,1995-12-15,262797249.0,...,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,"English, Français",Joe Johnston,"Robin Williams, Jonathan Hyde, Kirsten Dunst, ...",when sibling judy and peter discover an enchan...,1995,4.043035
2,0.0,"romance,comedy",15602,en,a family wedding reignites the ancient feud be...,11.712900,"Warner Bros., Lancaster Gate",United States of America,1995-12-22,0.0,...,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,English,Howard Deutch,"Walter Matthau, Jack Lemmon, Ann-Margret, Soph...",a family wedding reignites the ancient feud be...,1995,0.000000
3,16000000.0,"comedy,drama,romance",31357,en,cheated on mistreated and stepped on the women...,3.859495,Twentieth Century Fox Film Corporation,United States of America,1995-12-22,81452156.0,...,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,English,Forest Whitaker,"Whitney Houston, Angela Bassett, Loretta Devin...",cheated on mistreated and stepped on the woman...,1995,5.090760
4,0.0,comedy,11862,en,just when george banks has recovered from his ...,8.387519,"Sandollar Productions, Touchstone Pictures",United States of America,1995-02-10,76578911.0,...,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,English,Charles Shyer,"Steve Martin, Diane Keaton, Martin Short, Kimb...",just when george bank ha recovered from his da...,1995,0.000000


In [212]:
# Mostrar información del número de columnas, filas, tipos de datos y valores faltantes presentes en "merged"
merged.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45346 entries, 0 to 45345
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   budget                45346 non-null  float64       
 1   genres                42962 non-null  object        
 2   id                    45346 non-null  int64         
 3   original_language     45335 non-null  object        
 4   overview              45346 non-null  object        
 5   popularity            45346 non-null  float64       
 6   production_companies  33557 non-null  object        
 7   production_countries  39138 non-null  object        
 8   release_date          45346 non-null  datetime64[ns]
 9   revenue               45346 non-null  float64       
 10  runtime               45100 non-null  float64       
 11  status                45266 non-null  object        
 12  tagline               20383 non-null  object        
 13  title                

In [213]:
# % de datos faltantes respecto al total de los datos presentes en "merged"
total_cells = np.product(merged.shape)
total_missing = conteo_valores_faltantes.sum()
percent_missing = (total_missing / total_cells) * 100
print(f'{round(percent_missing, 2)}%')

5.29%


Dataset limpio en formato parquet

In [222]:
merged.to_csv('movies_clean.csv', index = False)
merged.to_parquet('movies_clean.parquet', index = False, compression='gzip')

Dataset para el consumo de la API

In [215]:
# Columnas necesarias para el consumo de la API
api = merged[['popularity' ,'release_date', 'title', 'vote_average', 'vote_count', 'actores',
              'release_year', 'director', 'return', 'budget', 'revenue']]

In [218]:
# Se exporta las columnas necesarias para el consumo de la API en formato .csv y .parquet
api.to_csv('api_consultas.csv', index = False)
api.to_parquet('api_consultas.parquet', index = False, compression='snappy')